- Ya tenemos los datos descargados y homogeneizados. Sin embargo, si abrimos algún activo al azar es probable que veamos datos en blanco (NULL), datos a cero, datos con error (NA). 
- Evidentemente no podremos trabajar mientras los datos contengan este tipo de problemas. Mejora la función de homogeneizar para arreglar de manera automática estos problemas. 
- Objetivo: Mejora la función homogeneizar para detectar y solventar los NULL y NA. 
- Tiempo objetivo: 30 minutos

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
from datetime import timedelta
from time import mktime
import time
from tqdm import tqdm
import math
from rcurl import get_curl
from io import BytesIO

In [2]:
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance  
    parameters: stock - short-handle identifier of the company    
    returns a tuple of header, crumb and cookie
    """   
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    
    with requests.session():
        
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))
        output=(header, crumb[0], website.cookies)
        
        return output        

In [3]:
def convert_to_unix(date):
    """
    converts date to unix timestamp    
    parameters: date - in format (yyyy-mm-dd)    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%Y-%m-%d')
    
    return int(mktime(datum.timetuple()))

In [4]:
def load_csv_data(stock, interval='1d', day_begin='20-03-2018', day_end='20-06-2018'):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    
    error1='404 Not Found: Timestamp data missing.'
    
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)   
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
        
        return website.text.split('\n')

In [5]:
def set_dates(x,ventana):
    
    """setup inicial de fechas,
    esta funcion devuelve la fecha de inicio
    y fin en string y formato YYYY-MM-DD
    Parametros:
    x=start o end
    ventana: entero referido al numero de dias del periodo"""
    
    from datetime import timedelta
    from datetime import datetime
    hoy=datetime.now()
    
    if x=='end':
        fecha_fin = str(hoy.now())
        fecha_fin = fecha_fin[0:10]
        return (fecha_fin)
    
    if x=='start':
        fecha_inicial=hoy-timedelta(ventana)
        fecha_inicial = str(fecha_inicial)
        fecha_inicial = fecha_inicial[0:10]
        return (fecha_inicial)
    
    else:
        print('inputs incorrectos. Ver docstring d ela funcion')   
    

In [6]:
def intentos_descarga_precios(stock_series,interval,fecha_inicial,fecha_fin, index_series, intentos):
    """funcion que en funcion del resultado de la descarga,
    devuelve un df con la informacion, y un codigo 0 'OK'
    o un codigo 1 de error y un DF vacio"""
    
    try:
        df = historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin, index_series)
        codigo = 'OK'
        e = 'OK'
        return df, codigo, e
    
    except Exception as e:
        print('El activo no tiene activos descargables')
        print('Mensaje de error: ',str(e))
        print('Intento numero ',str(intentos))
        time.sleep(3)
        df = pd.DataFrame ({'dato':[],
                            'Divisa':[]})
        codigo = 'ERROR'
        
        return df, codigo, e

In [7]:
def historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin, index_series):
    n=0
    x=0 
    for i in range(len(stock_series)):
        print("descargando "+stock_series[i]+' del indice '+index_series[i])
        try:
            DF=load_csv_data(stock_series[i], interval=interval, day_begin=fecha_inicial, day_end=fecha_fin)
            DF=pd.DataFrame(DF)
            DF = DF.iloc[:,0].str.split(",", n = 7, expand = True)
            DF.columns=DF.iloc[0,:]
            DF=DF.iloc[1:DF.shape[0],:]
            x=DF.shape[1]                 
            if x<2:
                 print("No nos hemos podido descargar el activo", benchmark)
            else:
                DF['stock'] = str(stock_series[i])
                DF['indice'] = str(index_series[i])
                DF=DF.dropna()
            
                if n==0:
                    dfacum=DF
                else:
                    dfacum=agrega_dataframes(DF,dfacum)    
                n=n+1
        except Exception as e:
            print("No nos hemos podido descargar el activo")
            print('key error:', e)
            next
        
    return dfacum


In [8]:
def agrega_dataframes(df,dfacum):
    """codigo que permite ir concatenando
    dataframes con estructura similar
    df:daframe a agregar
    dfacum: dataframe en donde se agregara"""
    
    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
    dfacum.reset_index(drop=True)
    return dfacum


In [9]:
def split_df_acum_by_stock(dfacum):
    """codigo que permite ir desconcatenando
    dataframes con estructura similar a partir de un df acumulado
    requiere que la clave de desagregacion sea la columna 'stock'
    dfacum: dataframe en se ha agregado"""
    
    activos_a_importar = dfacum.stock.unique()
    for activo in activos_a_importar:
        globals()[activo] = dfacum[dfacum['stock']==activo]

In [10]:
def vector_lab_dates(fecha_inicial,ventana):
    """con esta funcion se obtiene un listado de fechas
    entre lunes y viernes, a partir de una fecha inicial
    y una ventana"""
    dates = pd.date_range(fecha_inicial, periods=ventana, freq='B')    
    inicio = datetime.strptime(fecha_inicial, '%Y-%m-%d')
    dias =ventana
    dates=[]
    
    for days in range(dias):
        date = inicio + timedelta(days=days)
        if date.weekday() < 5:
            dates.append(date)
    return pd.DataFrame({'dates':dates})

In [11]:
def componentes_indice_ind_download_0(ticker):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    lista_dato = []
    
    #primero se obtiene la divisa
    url="https://es.finance.yahoo.com/quote/"+ticker+"/components/"
    soup  = requests.get(url)
    soup  = BeautifulSoup(soup.content, 'html.parser')
    
    name_box = soup.find('span', attrs={'data-reactid': '4'})
    name = name_box.text.strip()
    divisa=name[len(name)-3:len(name)]
    
    print(ticker,divisa)
    soup  = requests.get(url)
    soup  = BeautifulSoup(soup.content, 'html.parser')
    
    name_box = soup.find_all('td')
    name_boxccc=name_box[0]
    name_empresa=name_box[1]
    name_boxccc = name_boxccc.get_text(strip=True)
    name_empresa = name_empresa.get_text(strip=True)
    
    for i in range(0,len(name_box)):
        name_boxccc=name_box[i]
        name_boxccc = name_boxccc.get_text(strip=True)
        lista_dato.append(name_boxccc)
    df=pd.DataFrame ({'dato':lista_dato})
    df['Divisa'] = divisa
    
    return df

In [12]:
def intentos_descarga_indice(ticker, intentos):
    """funcion que en funcion del resultado de la descarga,
    devuelve un df con la informacion, y un codigo 0 'OK'
    o un codigo 1 de error y un DF vacio"""
    
    try:
        df = componentes_indice_ind_download_0(ticker)
        codigo = 'OK'
        e = 'OK'
        print('El índice ', ticker,' se ha descargado correctamente')
        return df, codigo, e
    
    except Exception as e:
        print('El índice ', ticker,' no tiene activos descargables')
        print('Mensaje de error: ',str(e))
        print('Intento numero ',str(intentos))
        
        time.sleep(3)
        df = pd.DataFrame ({'dato':[],
                            'Divisa':[]})
        codigo = 'ERROR'
        
        return df, codigo, e

In [13]:
def componentes_indice_ind_download_1(ticker):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""
    
    intentos = 1
    for i in range(3):
        
        intentos_descarga_indice_res = intentos_descarga_indice(ticker, intentos)
        df = intentos_descarga_indice_res[0]
        codigo = intentos_descarga_indice_res[1]
        e = intentos_descarga_indice_res[2]
        
        if codigo == 'OK':
            return df
        
        if (codigo == 'ERROR') & (intentos <=3):
            intentos = intentos + 1
            time.sleep(3)
            
        if intentos ==3:
            print('Mensaje de error: ',str(e))
            print("Tras varios intentos no nos hemos podido descargar",ticker,"Dejamos de intentarlo.")
            df = pd.DataFrame ({'dato':[],
                                'Divisa':[]})
            return df

In [14]:
def recupera_valor_secuencia(df, start, freq, encabezado):
    valor = pd.DataFrame(np.arange(start, len(df), freq))
    valor['key'] = 1
    valor.index = valor[0]
    valor = valor.drop(0, 1)
    valor = pd.merge(df, valor, left_index=True, right_index=True)
    valor.reset_index(drop=True, inplace=True)
    valor.rename(columns={"dato": encabezado},inplace=True)
    valor = valor.drop(['key'], 1)
    return valor

In [15]:
def componentes_indice_ind_download_2(tickers_de_indices):
    """En este tercer paso, con la informacion descargada,reconstruimos la tabla. 
    Tiene 6 columnas (simbolo, nombre de la empresa, último precio, cambio, cambio % y volumen.)
    # Nos interesa obtener el símbolo y el nombre de la empresa.
    # Ojo, queremos importar únicamente los activos que tengan cotización. 
    """
    intentos = 0
    
    dfacum = pd.DataFrame ({'Activo':[],
                            'Divisa':[],
                            'Empresa':[],
                            'Precio':[],
                            'Indice':[]})
    
    for indice in tickers_de_indices:
        df = componentes_indice_ind_download_1(indice)
        Activo = recupera_valor_secuencia(df, 0, 6, 'Activo')
        df = df.drop(['Divisa'],1)
        Empresa = recupera_valor_secuencia(df, 1, 6, 'Empresa')
        Precio = recupera_valor_secuencia(df, 2, 6, 'Precio')
        df = pd.merge(Activo, Empresa, left_index=True, right_index=True)
        df = pd.merge(df, Precio, left_index=True, right_index=True)
        df['Indice'] = indice
        df = df[df.loc[:,'Precio'] != '']
        dfacum = agrega_dataframes(dfacum, df)
        dfacum.reset_index(drop=True, inplace=True)
        
    return dfacum

In [16]:
def dataframe_date_adj(dfacum, dfdates):
    """ajustamos por  fechas.
    las pasamos como indice y armonizamos fechas.
    dates_df han pasado por una funcion en donde se han depurado sabados y domingos
    dfacum: dataframe con los campos, Adj Close, Close, Date, High, Low, Open, Volume, indice,stock 
    dates_df: : dataframe con las fechas que seran la muestra final"""
    
    dfacum.index=dfacum.Date
    dates_df.index=dates_df.dates
    dates_df.index = pd.to_datetime(dates_df.index).strftime('%d-%m-%Y')
    dfacum.index = pd.to_datetime(dfacum.index).strftime('%d-%m-%Y')                            
    df_date_adj=pd.merge(dates_df, dfacum,left_index=True, right_index=True)
    df_date_adj = df_date_adj.drop(['dates'],1)
    df_date_adj = df_date_adj.drop(['Date'],1)
    
    #Se ordena el DF por fechas (ya lo esta realmente por la manera construda, pero nos aseguramos)
    df_date_adj.sort_index(axis = 0)
    
    return df_date_adj

In [17]:
def remplaza_nulos_na(df, registros_a_depurar):
    """Con esta funcion se rellenan datos no informados.
    En un primer paso, los nulos por nas, y en un segundo paso,
    los na por el dato de la fecha previa, y para los primeros daros de la 
    df: dataframe con los campos, Adj Close, Close, Date, High, Low, Open, Volume, indice,stock 
    activos: dataframe con los nombres de los activos a los que se aplica"""
    
    #Se identifican los los nulos y se reemplazan con NA
    df[df == 'null'] = np.nan
    df.replace('', np.nan)
    df.replace(' ', np.nan)
    df.replace('-', np.nan)
    df.replace('NA', np.nan)
    
    for activo in registros_a_depurar:
        df[activo] = df[activo].fillna(method='pad')
        df[activo] = df[activo].fillna(method='bfill')
        
    return df

In [18]:
def limpieza_de_datos(dfacum, dates_df):
    """funcion que agrega las rutinas definidas en el apartado para la depuracion de datos, 
    en base a fechas, y gestion de ausencias.
    como inputs el dataframe global y el dataframe de fechas laborables"""
    
    activos_a_importar = dfacum.stock.unique()
    
    for activo in activos_a_importar:
        globals()[activo] = dataframe_date_adj(globals()[activo], dates_df)
        globals()[activo] = remplaza_nulos_na(globals()[activo], globals()[activo].columns)

In [19]:
tickers_de_indices=("%5EBFX","%5EBVSP","%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", 
                      "%5EMXX","%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI","%5EJKSE")
ventana = 500

In [20]:
dfacum = componentes_indice_ind_download_2(tickers_de_indices)

%5EBFX EUR
El índice  %5EBFX  se ha descargado correctamente
%5EBVSP BRL
El índice  %5EBVSP  se ha descargado correctamente
%5EDJI USD
El índice  %5EDJI  se ha descargado correctamente
%5EFCHI EUR
El índice  %5EFCHI  se ha descargado correctamente
%5EFTSE GBP
El índice  %5EFTSE  se ha descargado correctamente
%5EGDAXI EUR
El índice  %5EGDAXI  se ha descargado correctamente
%5EHSI HKD
El índice  %5EHSI  se ha descargado correctamente
%5EIBEX EUR
El índice  %5EIBEX  se ha descargado correctamente
%5EMXX MXN
El índice  %5EMXX  se ha descargado correctamente
%5EMERV  en
El índice  %5EMERV  se ha descargado correctamente
%5EOMXSPI SEK
El índice  %5EOMXSPI  se ha descargado correctamente
%5EOSEAX NOK
El índice  %5EOSEAX  se ha descargado correctamente
%5ESSMI CHF
El índice  %5ESSMI  se ha descargado correctamente
%5ESTI SGD
El índice  %5ESTI  se ha descargado correctamente
%5EJKSE IDR
El índice  %5EJKSE  se ha descargado correctamente


In [21]:
fecha_fin=set_dates('end',ventana)
fecha_inicial=set_dates('start',ventana)

print('la fecha de inicio es',fecha_inicial,' y la de fin es',fecha_fin)

la fecha de inicio es 2019-02-27  y la de fin es 2020-07-11


In [22]:
dates_df = vector_lab_dates(fecha_inicial, ventana)

In [23]:
dfacum = historic_prices_series(dfacum['Activo'],
                                            '1d', 
                                            fecha_inicial, 
                                            fecha_fin, 
                                            dfacum['Indice'])

descargando KDSI.JK del indice %5EJKSE
descargando BABP.JK del indice %5EJKSE
descargando PSKT.JK del indice %5EJKSE
descargando FMII.JK del indice %5EJKSE


KeyboardInterrupt: 

In [25]:
dfacum

,Activo,Divisa,Empresa,Indice,Precio
0,KDSI.JK,IDR,PT Kedawung Setia Industrial Tbk,%5EJKSE,"910,00"
1,BABP.JK,IDR,PT Bank MNC Internasional Tbk,%5EJKSE,"50,00"
2,PSKT.JK,IDR,PT Red Planet Indonesia Tbk,%5EJKSE,"50,00"
3,FMII.JK,IDR,PT Fortune Mate Indonesia Tbk,%5EJKSE,"800,00"
4,LPGI.JK,IDR,PT Lippo General Insurance Tbk,%5EJKSE,"3.750,00"
5,VOKS.JK,IDR,PT Voksel Electric Tbk,%5EJKSE,"200,00"
6,TRST.JK,IDR,PT Trias Sentosa Tbk,%5EJKSE,"382,00"
7,SULI.JK,IDR,PT SLJ Global Tbk,%5EJKSE,"50,00"
8,ABDA.JK,IDR,PT Asuransi Bina Dana Arta Tbk,%5EJKSE,"6.400,00"
9,ASDM.JK,IDR,PT Asuransi Dayin Mitra Tbk,%5EJKSE,"970,00"


In [26]:
#con este DF nos aseguramos que nos descargamos todos los registros por activo
muestra_stock = dfacum.groupby(['stock']).count()
muestra_stock

KeyError: 'stock'

Finalmente, vamos a obtener un DataFrame por activo

In [27]:
split_df_acum_by_stock(dfacum)

AttributeError: 'DataFrame' object has no attribute 'stock'

Recuperamos un ejemplo para comprobar que todo va bien

In [ ]:
globals().get('0003.HK').head()

Volcamos los datos en un DF homogeneizado, utilizando la columna de fechas como índice.

Completamos los datos con NA (días en los que no hay registrada una cotización, pero que otras empresas sí cotizaron) na.locf localiza los NA del DF y los sustituye por el valor de la fila anterior. Si la fila con NA es la 1ª deja el NA sin dar un error.

utilizo esta solucion
  
  http://pyciencia.blogspot.com/2015/04/trabajar-con-datos-nan-en-dataframe.html
  
_Con los valores vecinos
Reemplazar los NaN con el valor anterior o posterior del NaN: 'pad' para reemplazarlo con el valor anterior y 'bfill' con el posterior. Se reemplazan todos los NaN, pero se puede establecer un límite según la distancia de este con el último dato del dataframe. Esta distancia se especifica en limit:_

In [ ]:
limpieza_de_datos(dfacum, dates_df)

In [ ]:
globals().get('0003.HK').head()